In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
from transformers import pipeline
from datetime import datetime
from tqdm import tqdm

In [ ]:
# Инициализация анализатора тональности
analyzer = pipeline(
    "sentiment-analysis",
    model="blanchefort/rubert-base-cased-sentiment",
    tokenizer="blanchefort/rubert-base-cased-sentiment"
)

#Анализируем тональность текста с помощью модели
def analyze_sentiment(text):
    try:
        result = analyzer(str(text))[0]
        
        # Преобразуем метки модели в стандартный формат
        label_mapping = {
            'positive': 'positive',
            'neutral': 'neutral',
            'negative': 'negative'
        }
        
        return {
            'sentiment_score': float(result['score']),
            'sentiment_label': label_mapping.get(result['label'].lower(), 'neutral')
        }
    except Exception as e:
        print(f"Ошибка при анализе текста: {str(e)}")
        return {'sentiment_score': 0.0, 'sentiment_label': 'neutral'}

In [ ]:
#Получаем отзывы из базы данных и устанавливаем лимит, чтобы не грузить базу
def fetch_reviews(conn):
    try:
        query = """
            SELECT review_id, text 
            FROM reviews 
            WHERE review_id NOT IN (
                SELECT review_id FROM sentiment_analysis
            )
            LIMIT 100 
        """
        return pd.read_sql(query, conn)
    except Exception as e:
        print(f"Ошибка при загрузке отзывов: {str(e)}")
        return pd.DataFrame()

In [ ]:
#Сохраняем результаты анализа в базу данных
def save_results(conn, results):
    if not results:
        return
    
    # Подготавливаем данные для вставки
    try:
        with conn.cursor() as cur:
            data = [
                (
                    item['review_id'],
                    item['sentiment_score'],
                    item['sentiment_label'],
                    datetime.now()
                )
                for item in results
            ]
            
            # Вставляем данные пачками
            execute_batch(cur,
                """
                INSERT INTO sentiment_analysis 
                (review_id, sentiment_score, sentiment_label, created_at)
                VALUES (%s, %s, %s, %s)
                ON CONFLICT (review_id) DO UPDATE SET
                    sentiment_score = EXCLUDED.sentiment_score,
                    sentiment_label = EXCLUDED.sentiment_label,
                    created_at = EXCLUDED.created_at
                """,
                data,
                page_size=100
            )
            conn.commit()
            print(f"Сохранено {len(results)} результатов анализа")
    except Exception as e:
        conn.rollback()
        print(f"Ошибка при сохранении результатов: {str(e)}")

In [ ]:
#Запускаем главную функцию
def main():
    print("Начало анализа тональности отзывов...")
    
    try:
        with psycopg2.connect(**DB_CONFIG) as conn:
            #Получаем отзывы из БД
            reviews_df = fetch_reviews(conn)
            if reviews_df.empty:
                print("Нет новых отзывов для анализа")
                return
            
            print(f"Получено {len(reviews_df)} отзывов для анализа")
            
            #Анализируем тональность с прогресс-баром, чтобы видеть процесс
            results = []
            for _, row in tqdm(reviews_df.iterrows(), total=len(reviews_df), desc="Анализ отзывов"):
                analysis = analyze_sentiment(row['text'])
                results.append({
                    'review_id': row['review_id'],
                    **analysis
                })
            
            #Сохраняем результаты в БД
            save_results(conn, results)
            
    except Exception as e:
        print(f"Ошибка в основном процессе: {str(e)}")
    finally:
        print("Анализ завершен")

if __name__ == "__main__":
    main()

In [ ]:
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re
from psycopg2 import sql
from collections import Counter

In [ ]:
#Загружаем необходимые ресурсы NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#Проводим предварительную обработку текста
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    tokens = word_tokenize(text)
    
    stop_words = set(stopwords.words('russian'))
    tokens = [token for token in tokens if token not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [token for token in tokens if len(token) > 2]
    
    return tokens  

In [ ]:
#Извлекаем ключевые фразы из отзывов, топ-10 
def extract_key_phrases(reviews, top_n=10):
    all_tokens = []
    for review in reviews:
        all_tokens.extend(preprocess_text(review))
    
    if not all_tokens:
        return []
    
    #Считаем частоту слов и биграмм
    word_counts = Counter(all_tokens)
    
    #Извлекаем биграммы
    bigrams = [f"{all_tokens[i]} {all_tokens[i+1]}" 
              for i in range(len(all_tokens)-1)]
    bigram_counts = Counter(bigrams)
    
    #Объединяем результаты в Counter
    combined = word_counts + bigram_counts
    
    #Получаем топ-10 фраз
    return [phrase for phrase, _ in combined.most_common(top_n)]

In [ ]:
#Анализируем отзывы и сохраняем ключевын фразы
def process_products(conn, limit: int = 100):
    try:
        with conn.cursor() as cursor:
            #Получаем продукты с отзывами
            query = """
                SELECT r.product_id, array_agg(r.text) as reviews
                FROM reviews r
                WHERE r.text IS NOT NULL
                GROUP BY r.product_id
            """
            if limit:
                query += f" LIMIT {limit}"
                
            cursor.execute(query)
            products = cursor.fetchall()
            
            #Обрабатываем каждый продукт
            for product_id, reviews in products:
                phrases = extract_key_phrases(reviews)
                if phrases:
                    #Обновляем или добавляем записи в табличку
                    cursor.execute("""
                        INSERT INTO key_phrases (product_id, phrases, created_at)
                        VALUES (%s, %s, %s) """, (product_id, phrases, datetime.now()))
            
            conn.commit()
            print(f"Обработано продуктов: {len(products)}")
            
    except Exception as e:
        conn.rollback()
        print(f"Ошибка: {str(e)}")
        raise

In [ ]:
#Вызываем главную функцию
def main():
    conn = psycopg2.connect(**DB_CONFIG)
    try:
        process_products(conn, limit=100) #Устанавливаем лимит
    finally:
        conn.close()

if __name__ == "__main__":
    main()